In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r"./data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons

########################
n_hidden_3 = 256 
########################

n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])), #<-----
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),         #<-----
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.dropout(layer_2, 0.5)
    layer_3 = tf.matmul(layer_2, weights['h3']) + biases['b3'] #<----
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)


# Define loss and optimizer
beta = 0.01
beta2 = 0.05
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

loss_op = tf.reduce_mean(loss_op + 
                         beta*tf.nn.l2_loss(weights['h1'])+
                         beta2*tf.nn.l2_loss(weights['h2']))

# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate*100)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
            
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost=36698.185129617
Epoch: 0002 cost=14240.677443182
Epoch: 0003 cost=9241.055856712
Epoch: 0004 cost=7387.883592862
Epoch: 0005 cost=6459.049491300
Epoch: 0006 cost=5894.167360174
Epoch: 0007 cost=5501.239261364
Epoch: 0008 cost=5199.774666193
Epoch: 0009 cost=4962.639156605
Epoch: 0010 cost=4772.769564986
Epoch: 0011 cost=4612.482223011
Epoch: 0012 cost=4477.372581232
Epoch: 0013 cost=4358.910283203
Epoch: 0014 cost=4255.536077770
Epoch: 0015 cost=4165.200144709
Optimization Finished!
Accuracy: 0.8581


In [4]:
import matplotlib.pyplot as plt